In [ ]:
!pip install crewai

In [2]:
import os
from crewai import Agent, Task, Crew, Process

os.environ["OPENAI_API_KEY"] = ""

In [3]:
import pandas as pd
df = pd.read_csv('telecom_churn.csv')

In [4]:
df.head(10)

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.70,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.70,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.00,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.00,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.00,3,166.7,113,41.0,7.42,10.1
5,0,118,0,0,0.00,0,223.4,98,57.0,11.03,6.3
6,0,121,1,1,2.03,3,218.2,88,87.3,17.43,7.5
7,0,147,0,0,0.00,0,157.0,79,36.0,5.16,7.1
8,0,117,1,0,0.19,1,184.5,97,63.9,17.58,8.7
9,0,141,0,1,3.02,0,258.6,84,93.2,11.10,11.2


In [5]:
df[['Churn', 'AccountWeeks', 'ContractRenewal']].sample(100).reset_index(drop = True).to_csv('100_samples_telecom_churn.csv', index = False)

Intiate the tool to read the CSV files

In [6]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.agents import Tool

loader = CSVLoader(file_path="/content/100_samples_telecom_churn.csv")

load_tool = Tool(
  name="Data Loader Tool",
  func=loader.load,
  description="Load the CSV data from the source",
)

Creating Blog Posts from the Churn Dataset.

In [7]:
# Define your agents with roles and goals
analyst = Agent(
  role='Senior Data Analyst',
  goal='Load the dataset and provide insights from the data',
  backstory="""You work at a telecom company.
  Your expertise lies in identifying trends and analysis from the available data.
  You have a knack for dissecting complex data and presenting
  actionable insights.""",
  verbose=True,
  allow_delegation=False
)

writer = Agent(
  role='Data Content Strategist',
  goal='Craft compelling content from the data',
  backstory="""You are a renowned data writer, known for
  your insightful and engaging articles from the dataset.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=False,
)

In [9]:
from crewai.task import Task

In [12]:
task1 = Task(
  description="""Conduct a comprehensive analysis from the CSV dataset by using the tools provided.
  You only need to run the tools and without providing any input, as the tools already have the source data path embedded to it.
  Identify any analysis that related to why churn is happening.
  Your final answer MUST be a full analysis report""",
  agent=analyst,
  tools=[load_tool],
  expected_output="""Your analysis report should include the following sections:
  - Introduction
  - Data Exploration
  - Churn Analysis
  - Conclusion"""
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlight your finding for the others.
  Your post should be informative, especially for casual people.
  Make it profoessional and avoid complex words so it doesn't sound like AI.
  Your final answer MUST be the full blog post of at least 3 paragraphs.""",
  agent=writer,
  tools=[load_tool],
  expected_output="Provide a comprehensive analysis report on the causes of churn."
)

In [14]:
task2

Task(description=Using the insights provided, develop an engaging blog
  post that highlight your finding for the others.
  Your post should be informative, especially for casual people.
  Make it profoessional and avoid complex words so it doesn't sound like AI.
  Your final answer MUST be the full blog post of at least 3 paragraphs., expected_output=Provide a comprehensive analysis report on the causes of churn.)

In [15]:
task1

Task(description=Conduct a comprehensive analysis from the CSV dataset by using the tools provided.
  You only need to run the tools and without providing any input, as the tools already have the source data path embedded to it.
  Identify any analysis that related to why churn is happening.
  Your final answer MUST be a full analysis report, expected_output=Your analysis report should include the following sections:
  - Introduction
  - Data Exploration
  - Churn Analysis
  - Conclusion)

In [13]:
crew = Crew(
  agents=[analyst, writer],
  tasks=[task1, task2],
  verbose=2,
)

result = crew.kickoff()

 [DEBUG]: == Working Agent: Senior Data Analyst
 [INFO]: == Starting Task: Conduct a comprehensive analysis from the CSV dataset by using the tools provided.
  You only need to run the tools and without providing any input, as the tools already have the source data path embedded to it.
  Identify any analysis that related to why churn is happening.
  Your final answer MUST be a full analysis report


> Entering new CrewAgentExecutor chain...


NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4` does not exist or you do not have access to it. Learn more: https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}